# This code was used to parse multiple datasets in the Text Age project -

The process is split into multiple centuries and a few other opensource datasets are added into creating the final code

In [5]:
# Text age parser
from bs4 import BeautifulSoup
#import datasets
from datetime import datetime
import os

dt_format = "%Y"
base_path = r"C:\Users\rhira\Documents\TextAge\17001800\dataset"
files = os.listdir(base_path)
print(files)

['1508_Richard-rolle_The-remedy-against-the-troubles-of-temptations_book.xml', '1508_William-Dunbar_The-ballade-of-ane-right-noble-victorius_book.xml', '1511_Henry-watson_The-noble-history-of-King-Ponthus_book.xml', '1530_Edward-Gosynhyll_dialogue-betwene-the-comen-secretary-and-Ielowsy-touchynge-the-vnstablenes-of-harlottes_letter.xml', '1530_Richard-ullerston_A-compendious-olde-treatyse_book.xml', '1531_Elyyot-Thomas_The-boke-named-The-Gouernour_book.xml', '1532_Hervet-gentian_Xenophons-treatise-of-ousholde_treaty.xml', '1535_George-joye_A-compendyouse-somme-of-the-very-Christen_book.xml', '1535_Joye-George_An-apolgye-to-w.-Tindale_letter.xml', '1542_Heinrich-cornelius_A-treatise-of-the-nobilitie-and-excellencye-of-vvoman-kynde_treaty.xml', '1549_William-Baldwin_The-canticles-or-balades-of-Salomon_book.xml', '1550_Richard-sherry_A-treatise-of-schemes-&-tropes_book.xml', '1550_unknown_Pyers-plowmans-exhortation_book.xml', '1551_Henry-wingfield_A-compendious-or-short-treatise_treaty.xm

In [6]:
import pandas as pd
df = pd.DataFrame(columns=["text","category","date","head","title","author"])

In [3]:
from datasets import load_dataset

dataset = load_dataset("biglam/lampeter_corpus")

c:\Users\rhira\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 120/120 [00:00<00:00, 611.85 examples/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'date', 'genre', 'head', 'title'],
        num_rows: 120
    })
})

In [6]:
df_pandas = dataset["train"].to_pandas()
df_pandas.head()

,id,text,date,genre,head,title
0,EcA1641,To contend with men that deny their owne publi...,1641-01-01,Economy,"A Reply to a most untrue Relation made, and se...",A reply to a most untrue relation [...] by cer...
1,EcA1652,IT hath been a thing for many years generally ...,1652-01-01,Economy,"THE ADVOCATE: OR, A NARRATIVE Of the state and...",The Advocate.
2,EcA1668,"I Presume, there are few Englishmen so disloya...",1668-01-01,Economy,\nPropos. 1.\n\nIt will supply his Majesties p...,"A discourse ,shewing the many advantages which..."
3,EcA1676,HAving lately seen a Pamphlet under the Title ...,1676-01-01,Economy,An Enquiry into the Grounds of a late Pamphlet...,Is not the hand of Joab in all this? Or An enq...
4,EcA1681,FOrasmuch as there is a very great Complaint i...,1681-01-01,Economy,SECT. I. \nThe Introduction.,The Trade of England Revived: And the Abuses t...


In [7]:
df_pandas.to_csv("lampenter.csv")

In [ ]:
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd

def extract_data(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    title = root.find(".//title").text
    author = root.find(".//author").text

    # Extract 4-digit year from speechPubDate within the range 1500 to 1900
    raw_speech_pub_date = root.find(".//speechPubDate").text
    match = re.search(r'\b(15\d{2}|16\d{2}|17\d{2}|18\d{2}|19\d{2})\b', raw_speech_pub_date)
    speech_pub_date = match.group() if match else None

    dialogue_elements = root.findall(".//dialogueText//dialogue")
    non_speech_elements = root.findall(".//dialogueText//nonSpeech")

    dialogue_content = ' '.join([''.join(dialogue.itertext()).strip() for dialogue in dialogue_elements])
    non_speech_content = ' '.join([''.join(non_speech.itertext()).strip() for non_speech in non_speech_elements])

    combined_content = dialogue_content + ' ' + non_speech_content

    data = {'Title': [title], 'Author': [author], 'SpeechPubDate': [speech_pub_date], 'Content': [combined_content]}
    df = pd.DataFrame(data)
    return df

def process_folder(folder_path):
    all_dfs = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".xml"):
            xml_file_path = os.path.join(folder_path, filename)
            df = extract_data(xml_file_path)
            all_dfs.append(df)

    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        return combined_df
    else:
        print("No XML files found in the folder.")
        return None

# Replace 'your_folder_path' with the actual path to your folder containing XML files
folder_path = 'english_dialogues'
combined_dataframe = process_folder(folder_path)

# Display the combined DataFrame
if combined_dataframe is not None:
    pd.set_option('display.max_colwidth', None)
    print(combined_dataframe)


In [40]:
combined_dataframe["year"] = combined_dataframe["SpeechPubDate"]
combined_dataframe["year"]

0      1599
1      1594
2      1584
3      1595
4      1595
       ... 
172    1752
173    1729
174    1744
175    1726
176    1725
Name: year, Length: 177, dtype: object

In [ ]:
#files = ["1570_John-Drout_The-pityfull-histori-of-two-louing-Italians_book_book.xml"]
for filepath in files:
    with open(base_path+"/"+filepath, encoding="utf-8") as f:
            
            split = filepath.split("_")
            date = split[0]
            author = split[1]
            title = split[2]
            category = split[3]

            soup = BeautifulSoup(f, features="xml")
            for entry in soup.find_all("TEI"):
                text_parts = []
                #title_with_id = entry.teiHeader.fileDesc.titleStmt.title.text
                #date = entry.teiHeader.fileDesc.edition.date
                #print(title_with_id, date)
                #id, title = title_with_id.split(":", maxsplit=1)
                #id = id.strip()
                #title = title.strip()
                #date = datetime.strptime(id[-4:], dt_format)
                content = entry.find("text")
                #print(content)
                head = content.find("body").find("head")
                
                if head:
                    head = head.text
                else:
                    head = ""
                body_parts = content.find("body").find_all("p")
                if len(str(body_parts)) < 100:
                     body_parts = content.find("body").find_all("l")
                for body_part in body_parts:
                    text_parts.append(body_part.text)
                full_text = " ".join(text_parts)
                #print(filepath, len(full_text))

                #genre = _CLASS_MAP[id[0]]
                data_point = {
                    "text": full_text,
                    "category": category,
                    "date": date,
                    "head": head,
                    "title": title,
                    "author":author
                }
                df = df.append(data_point, ignore_index=True)
                #print(data_point)

In [9]:
df.category.value_counts()

book.xml                                                          38
report.xml                                                        21
novel.xml                                                         18
letter.xml                                                        18
poem.xml                                                          16
sermon.xml                                                        12
history.xml                                                        7
play.xml                                                           6
drama.xml                                                          5
travel.xml                                                         3
fiction.xml                                                        3
speech.xml                                                         3
poems.xml                                                          3
Drama.xml                                                          3
treaty.xml                        

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      180 non-null    object
 1   category  180 non-null    object
 2   date      180 non-null    object
 3   head      180 non-null    object
 4   title     180 non-null    object
 5   author    180 non-null    object
dtypes: object(6)
memory usage: 8.6+ KB


# Lampenter

In [131]:
x = pd.read_csv("lampenter.csv")
x["date"].head()

0    1641-01-01
1    1652-01-01
2    1668-01-01
3    1676-01-01
4    1681-01-01
Name: date, dtype: object

In [132]:
x.columns

Index(['Unnamed: 0', 'id', 'text', 'date', 'genre', 'head', 'title'], dtype='object')

In [133]:
lst = x["date"].to_list()
yr = [int(x.split("-")[0]) for x in lst if x]

In [134]:
x["year"] = yr

In [137]:
lampenter_1700 = x[x["year"]>1700]
lampenter_1700.to_csv("17001800.csv")

# Gefa

In [81]:
gefa = pd.read_csv("GEFA_18001900.csv")
gefa.columns

Index(['Unnamed: 0', 'author_tag', 'title', 'subjects', 'subjectcount', 'pd',
       'dob', 'dod', 'edp', 'decade', 'century', 'text'],
      dtype='object')

In [83]:
gefa.drop("text",axis=1)

,Unnamed: 0,author_tag,title,subjects,subjectcount,pd,dob,dod,edp,decade,century
0,11,MÃ¼hlbachLLuise,Napoleon and BlÃ¼cher: An Historical Novel,"Napoleon I, Emperor of the French, 1769-1821 -- Fiction; BlÃ¼cher, Gebhard Leberecht von, 1742-1819 -- Fiction; Germany -- History -- 1806-1815 -- Fiction",3,1814,1814,1873,1843,1840,19
1,13,MÃ¼hlbachLLuise,Louisa of Prussia and Her Times: A Historical Novel,"Napoleon I, Emperor of the French, 1769-1821 -- Fiction; Luise, Queen, consort of Frederick William III, King of Prussia, 1776-1810 -- Fiction; Prussia (Germany) -- History -- Frederick William III, 1797-1840 -- Fiction; Napoleonic Wars, 1800-1815 -- Campaigns -- Germany -- Prussia -- Fiction",4,1867,1814,1873,1843,1840,19
2,18,MÃ¼hlbachLLuise,Frederick the Great and His Court,"Frederick II, King of Prussia, 1712-1786 -- Fiction; Prussia (Germany) -- History -- Frederick II, 1740-1786 -- Fiction; Courts and courtiers -- Fiction",3,1866,1814,1873,1843,1840,19
3,93,MÃ¼hlbachLLuise,Andreas Hofer: An Historical Novel,"Hofer, Andreas, 1767-1810 -- Fiction; Tyrol (Austria) -- History -- Uprising of 1809 -- Fiction",2,1868,1814,1873,1843,1840,19
4,125,MÃ¼hlbachLLuise,Marie Antoinette and Her Son,"Historical fiction; Marie Antoinette, Queen, consort of Louis XVI, King of France, 1755-1793 -- Fiction; Louis XVII, of France, 1785-1795 -- Fiction",3,1866,1814,1873,1843,1840,19
...,...,...,...,...,...,...,...,...,...,...,...
8545,12865,AlexanderMrs,A Crooked Path: A Novel,Inheritance and succession -- Fiction; England -- Social life and customs -- Fiction,2,1889,1825,1902,1863,1860,19
8546,12874,AlexanderMrs,Ralph Wilton's weird,Inheritance and succession -- Fiction; Mate selection -- Fiction; Courtship -- Fiction; Social classes -- England -- Fiction,4,1875,1825,1902,1863,1860,19
8547,12883,AlexanderMrs,At Bay,Fiction,1,1854,1825,1902,1863,1860,19
8548,12888,AlexanderMrs,Women Novelists of Queen Victoria's Reign: A Book of Appreciations,"Novelists, English -- 19th century -- Biography; Women and literature -- Great Britain -- History -- 19th century; English fiction -- 19th century -- History and criticism; English fiction -- Women authors -- History and criticism; Women novelists, English -- Biography",5,1897,1825,1902,1863,1860,19


In [103]:
egefa = gefa[gefa["pd"]<1899]
ngefa = gefa[gefa["pd"]>1899]

In [105]:
ngefa.to_csv("19002000.csv")

## Dividing lampenter

In [130]:
lampenter = pd.read_csv("lampenter.csv")
lampenter.date.head(1)

0    1641-01-01
Name: date, dtype: object

# Century wise cleaning and the combining the files


## 1500 - 1600

In [31]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

df_1500 = pd.DataFrame(columns=["text","category","date","title","author"])

lst = os.listdir(r"C:\Users\rhira\Documents\TextAge\15001600")
for nam in lst:
    yr, aut, tit, typ = nam.split("_")
    #print(yr, aut, tit, type)
    
    xml_file_path = r'C:\Users\rhira\Documents\TextAge\15001600'+"\\"+nam
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Define a namespace dictionary for convenience
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Function to safely extract text content from an element
    def get_element_text(element, xpath, namespaces=ns):
        result = element.find(xpath, namespaces=namespaces)
        return result.text.strip() if result is not None and result.text is not None else None

    # Function to recursively concatenate text content from an element and its children
    def get_all_text(element, namespaces=ns, exclude_tags=None):
        result = ''
        if element.text and (exclude_tags is None or all(tag not in element.tag for tag in exclude_tags)):
            result += element.text.strip()
        for child in element:
            result += get_all_text(child, exclude_tags=exclude_tags)
            if child.tail:
                result += child.tail.strip()
        return result
    
    exclude_tags = {'titleStmt', 'licence', 'author', 'respStmt'}
    
    all_text = get_all_text(root, exclude_tags=exclude_tags)
    
    # text","category","date","head","title","author"
    dct = {"text":all_text,
           "date": yr, 
           "author":aut, 
           "title":tit, 
           "category":typ}
    
    df_dictionary = pd.DataFrame([dct])
    df_1500 = pd.concat([df_1500, df_dictionary], ignore_index=True)


# Exclude specific elements from being included in the te

df_1500.head()
    

,text,category,date,title,author
0,[The remedy against the troubles of temptation...,book.xml,1508,The-remedy-against-the-troubles-of-temptations,Richard-rolle
1,The ballade of ane right noble victorius & myg...,book.xml,1508,The-ballade-of-ane-right-noble-victorius,William-Dunbar
2,[The noble history of King Ponthus.]Ponthus et...,book.xml,1511,The-noble-history-of-King-Ponthus,Henry-watson
3,Here begynneth a dialogue betwene the comen se...,letter.xml,1530,dialogue-betwene-the-comen-secretary-and-Ielow...,Edward-Gosynhyll
4,"A compendious olde treatyse, shewynge howe tha...",book.xml,1530,A-compendious-olde-treatyse,Richard-ullerston


In [32]:
df_1500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      50 non-null     object
 1   category  50 non-null     object
 2   date      50 non-null     object
 3   title     50 non-null     object
 4   author    50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB


## 1600-1700

In [25]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

df_1600 = pd.DataFrame(columns=["text","category","date","title","author"])

lst = os.listdir(r"C:\Users\rhira\Documents\TextAge\16001700")

lst.remove("16001700.csv")
for nam in lst:
    #print(nam)
    yr, aut, tit, typ = nam.split("_")
    #print(yr, aut, tit, type)
    
    xml_file_path = r'C:\Users\rhira\Documents\TextAge\16001700'+"\\"+nam
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Define a namespace dictionary for convenience
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Function to safely extract text content from an element
    def get_element_text(element, xpath, namespaces=ns):
        result = element.find(xpath, namespaces=namespaces)
        return result.text.strip() if result is not None and result.text is not None else None

    # Function to recursively concatenate text content from an element and its children
    def get_all_text(element, namespaces=ns, exclude_tags=None):
        result = ''
        if element.text and (exclude_tags is None or all(tag not in element.tag for tag in exclude_tags)):
            result += element.text.strip()
        for child in element:
            result += get_all_text(child, exclude_tags=exclude_tags)
            if child.tail:
                result += child.tail.strip()
        return result
    
    exclude_tags = {'titleStmt', 'licence', 'author', 'respStmt'}
    
    all_text = get_all_text(root, exclude_tags=exclude_tags)
    
    # text","category","date","head","title","author"
    dct = {"text":all_text,
           "date": yr, 
           "author":aut, 
           "title":tit, 
           "category":typ}
    
    df_dictionary = pd.DataFrame([dct])
    df_1600 = pd.concat([df_1600, df_dictionary], ignore_index=True)


# Exclude specific elements from being included in the te


In [26]:
df_1600.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      70 non-null     object
 1   category  70 non-null     object
 2   date      70 non-null     object
 3   title     70 non-null     object
 4   author    70 non-null     object
dtypes: object(5)
memory usage: 2.9+ KB


In [28]:

df_1600_extra = pd.read_csv(r"C:\Users\rhira\Documents\TextAge\16001700\16001700.csv")
df_1600_extra.drop(["Unnamed: 0.1","Unnamed: 0","id","date"],axis=1, inplace=True)
df_1600_extra.rename(columns={"genre": "category", "year": "date","head": "author"}, inplace=True)
df_1600_extra.head()

,text,category,author,title,date
0,To contend with men that deny their owne publi...,Economy,"A Reply to a most untrue Relation made, and se...",A reply to a most untrue relation [...] by cer...,1641
1,IT hath been a thing for many years generally ...,Economy,"THE ADVOCATE: OR, A NARRATIVE Of the state and...",The Advocate.,1652
2,"I Presume, there are few Englishmen so disloya...",Economy,\nPropos. 1.\n\nIt will supply his Majesties p...,"A discourse ,shewing the many advantages which...",1668
3,HAving lately seen a Pamphlet under the Title ...,Economy,An Enquiry into the Grounds of a late Pamphlet...,Is not the hand of Joab in all this? Or An enq...,1676
4,FOrasmuch as there is a very great Complaint i...,Economy,SECT. I. \nThe Introduction.,The Trade of England Revived: And the Abuses t...,1681


In [29]:
df_1600 = pd.concat([df_1600, df_1600_extra], ignore_index=True)
df_1600.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      142 non-null    object
 1   category  142 non-null    object
 2   date      142 non-null    object
 3   title     142 non-null    object
 4   author    136 non-null    object
dtypes: object(5)
memory usage: 5.7+ KB


# 1700 1800

In [33]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

df_1700 = pd.DataFrame(columns=["text","category","date","title","author"])

lst = os.listdir(r"C:\Users\rhira\Documents\TextAge\17001800\dataset")

lst.remove("17001800.csv")
for nam in lst:
    #print(nam)
    yr, aut, tit, typ = nam.split("_")
    #print(yr, aut, tit, type)
    
    xml_file_path = r'C:\Users\rhira\Documents\TextAge\17001800\dataset'+"\\"+nam
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Define a namespace dictionary for convenience
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Function to safely extract text content from an element
    def get_element_text(element, xpath, namespaces=ns):
        result = element.find(xpath, namespaces=namespaces)
        return result.text.strip() if result is not None and result.text is not None else None

    # Function to recursively concatenate text content from an element and its children
    def get_all_text(element, namespaces=ns, exclude_tags=None):
        result = ''
        if element.text and (exclude_tags is None or all(tag not in element.tag for tag in exclude_tags)):
            result += element.text.strip()
        for child in element:
            result += get_all_text(child, exclude_tags=exclude_tags)
            if child.tail:
                result += child.tail.strip()
        return result
    
    exclude_tags = {'titleStmt', 'licence', 'author', 'respStmt'}
    
    all_text = get_all_text(root, exclude_tags=exclude_tags)
    
    # text","category","date","head","title","author"
    dct = {"text":all_text,
           "date": yr, 
           "author":aut, 
           "title":tit, 
           "category":typ}
    
    df_dictionary = pd.DataFrame([dct])
    df_1700 = pd.concat([df_1700, df_dictionary], ignore_index=True)


# Exclude specific elements from being included in the te


In [34]:

df_1700_extra = pd.read_csv(r"C:\Users\rhira\Documents\TextAge\17001800\dataset\17001800.csv")
df_1700_extra.drop(["Unnamed: 0.1","Unnamed: 0","id","date"],axis=1, inplace=True)
df_1700_extra.rename(columns={"genre": "category", "year": "date","head": "author"}, inplace=True)
df_1700_extra.head()

,text,category,author,title,date
0,\n\nThe Influence of the Bank considerable. NO...,Economy,THE Introduction.,Remarks upon the Bank of England [...],1705
1,THERE has been a Paper already offered to the ...,Economy,"SOME OBSERVATIONS, SHEWING The DANGER of Losin...","Some observations, shewing the danger of losin...",1714
2,THE Miscarriage of the Naval StoreBill last Se...,Economy,A LETTER TO A Member of Parliament.,"A letter to a member of parliament, concerning...",1720
3,IT is not possible that the Island of Barbadoe...,Economy,"\nREMARKS Upon a BOOK, Entituled, The Present ...","Remarks upon a book, entituled, the present st...",1731
4,"WHEN I see a Proposal for raising of Money, wi...",Economy,"THE Injustice, Dishonour AND Ill-Policy OF Bre...","Wednesday club-law: or, The injustice, dishono...",1717


In [35]:
df_1700 = pd.concat([df_1700, df_1700_extra], ignore_index=True)
df_1700.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      104 non-null    object
 1   category  104 non-null    object
 2   date      104 non-null    object
 3   title     104 non-null    object
 4   author    96 non-null     object
dtypes: object(5)
memory usage: 4.2+ KB


## 1800 1900

In [67]:
df_1800 = pd.DataFrame(columns=["text","category","date","title","author"])

lst = os.listdir(r"C:\Users\rhira\Documents\TextAge\18001900")

lst.remove("18001900.csv")

for nam in lst:
    #print(nam)
    #yr, aut, tit, typ = nam.split("_")
    typ, yr, fid = nam.split("_")
    f = open(r"C:\Users\rhira\Documents\TextAge\18001900"+"\\"+nam, "r")
    txt = f.read()

    dct = {"text":txt[8:],
           "date": yr, 
           "author":txt[8:50], 
           "title":fid, 
           "category":typ}
    
    df_dictionary = pd.DataFrame([dct])
    df_1800 = pd.concat([df_1800, df_dictionary], ignore_index=True)
df_1800.head(10)

,text,category,date,title,author
0,The author is indebted to one of the novels of...,fic,1817,8550.txt,The author is indebted to one of the novel
1,"Dramatis Person . Alexius , Emperour of Greece...",fic,1827,8950.txt,"Dramatis Person . Alexius , Emperour of Gr"
2,""" Oh Love ! what is it in this world of ours ....",fic,1830,8750.txt,""" Oh Love ! what is it in this world of ou"
3,"TO MRS . HAMBLIN , THIS DRAMATIC TRIFLE . ( Th...",fic,1831,8850.txt,"TO MRS . HAMBLIN , THIS DRAMATIC TRIFLE ."
4,THE PARTISAN LEADER ; A TALE OF THE FUTURE . B...,fic,1836,7350.txt,THE PARTISAN LEADER ; A TALE OF THE FUTURE
5,PROLOGUE . DELIVERED BY MRS . PRESTON . This n...,fic,1836,8650.txt,PROLOGUE . DELIVERED BY MRS . PRESTON . Th
6,"CHARCOAL SKETCHES ; OR , SCENES IN A METROPOLI...",fic,1838,7250.txt,"CHARCOAL SKETCHES ; OR , SCENES IN A METRO"
7,"Chivers , T. H. ( Thomas Holley ) , 1809-1858 ...",fic,1845,7450.txt,"Chivers , T. H. ( Thomas Holley ) , 1809-1"
8,"Hoffman , Charles Fenno , 1806-1884 : AFTERTHO...",fic,1847,7650.txt,"Hoffman , Charles Fenno , 1806-1884 : AFTE"
9,"CAST OF CHARACTERS . Uncle Jeff , a mischief-m...",fic,1868,9550.txt,"CAST OF CHARACTERS . Uncle Jeff , a mischi"


In [68]:
df_1800_extra = pd.read_csv(r"C:\Users\rhira\Documents\TextAge\18001900\18001900.csv")
df_1800_extra.drop(["Unnamed: 0.1","Unnamed: 0","subjectcount","dob","dod","edp","century","decade"],axis=1, inplace=True)
# text	category	date	title	author
df_1800_extra.rename(columns={"subjects": "category", "pd": "date","author_tag": "author"}, inplace=True)

df_1800 = pd.concat([df_1800, df_1800_extra], ignore_index=True)
df_1800.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4449 entries, 0 to 4448
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      4449 non-null   object
 1   category  4449 non-null   object
 2   date      4449 non-null   object
 3   title     4449 non-null   object
 4   author    4449 non-null   object
dtypes: object(5)
memory usage: 173.9+ KB


## 1900 2000

In [69]:
df_1900 = pd.DataFrame(columns=["text","category","date","title","author"])

lst = os.listdir(r"C:\Users\rhira\Documents\TextAge\19002000")

lst.remove("19002000.csv")

for nam in lst:
    #print(nam)
    #yr, aut, tit, typ = nam.split("_")
    typ, yr, fid = nam.split("_")
    f = open(r"C:\Users\rhira\Documents\TextAge\19002000"+"\\"+nam, "r")
    txt = f.read()

    dct = {"text":txt[8:],
           "date": yr, 
           "author":txt[8:50], 
           "title":fid, 
           "category":typ}
    
    df_dictionary = pd.DataFrame([dct])
    df_1900 = pd.concat([df_1900, df_dictionary], ignore_index=True)
df_1900.head(10)

,text,category,date,title,author
0,( This file was produced from scans of public ...,fic,1900,3850.txt,( This file was produced from scans of pub
1,"\nA kitchen . At the right an ironing-board , ...",fic,1910,13150.txt,\nA kitchen . At the right an ironing-board
2,JIMSY *** Produced by Annie McGuire Jimsy The ...,fic,1915,5250.txt,JIMSY *** Produced by Annie McGuire Jimsy
3,THE VALLEY OF VISION A BOOK OF ROMANCE AND SOM...,fic,1919,1550.txt,THE VALLEY OF VISION A BOOK OF ROMANCE AND
4,HE DRUMS OF JEOPARDY By Harold MacGrath CHAPTE...,fic,1920,550.txt,HE DRUMS OF JEOPARDY By Harold MacGrath CH
5,\nVIOLETTE A STORY BY VINCENT SHEEAN T HE pass...,fic,1930,23150.txt,\nVIOLETTE A STORY BY VINCENT SHEEAN T HE p
6,"\nVr ) 411 , - THE MAN WHO KNEW EVERYBODY A ST...",fic,1933,23250.txt,"\nVr ) 411 , - THE MAN WHO KNEW EVERYBODY A"
7,"A Sequel to "" Seed of the Arctic Ice "" Under A...",fic,1933,5650.txt,"A Sequel to "" Seed of the Arctic Ice "" Und"
8,\nLADY IN TROUBLE A STORY BY MARGERY SHARP ry'...,fic,1937,23350.txt,\nLADY IN TROUBLE A STORY BY MARGERY SHARP
9,\nWILD LINES : SWIMMING HOLE SCENE View page i...,fic,1938,16350.txt,\nWILD LINES : SWIMMING HOLE SCENE View pag


In [70]:
df_1900_extra = pd.read_csv(r"C:\Users\rhira\Documents\TextAge\19002000\19002000.csv")
df_1900_extra.drop(["Unnamed: 0.1","Unnamed: 0","subjectcount","dob","dod","edp","century","decade"],axis=1, inplace=True)
# text	category	date	title	author
df_1900_extra.rename(columns={"subjects": "category", "pd": "date","author_tag": "author"}, inplace=True)

df_1900 = pd.concat([df_1900, df_1900_extra], ignore_index=True)
df_1900.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4555 entries, 0 to 4554
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      3121 non-null   object
 1   category  3121 non-null   object
 2   date      3121 non-null   object
 3   title     3121 non-null   object
 4   author    3121 non-null   object
dtypes: object(5)
memory usage: 178.1+ KB


In [77]:
df_1900.head()

,text,category,date,title,author
0,( This file was produced from scans of public ...,fic,1900,3850.txt,( This file was produced from scans of pub
1,"\nA kitchen . At the right an ironing-board , ...",fic,1910,13150.txt,\nA kitchen . At the right an ironing-board
2,JIMSY *** Produced by Annie McGuire Jimsy The ...,fic,1915,5250.txt,JIMSY *** Produced by Annie McGuire Jimsy
3,THE VALLEY OF VISION A BOOK OF ROMANCE AND SOM...,fic,1919,1550.txt,THE VALLEY OF VISION A BOOK OF ROMANCE AND
4,HE DRUMS OF JEOPARDY By Harold MacGrath CHAPTE...,fic,1920,550.txt,HE DRUMS OF JEOPARDY By Harold MacGrath CH


In [71]:
fin_df = pd.concat([df_1500, df_1600,df_1700,df_1800,df_1900], ignore_index=True)
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9300 entries, 0 to 9299
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      7866 non-null   object
 1   category  7866 non-null   object
 2   date      7866 non-null   object
 3   title     7866 non-null   object
 4   author    7852 non-null   object
dtypes: object(5)
memory usage: 363.4+ KB


In [72]:
fin_df.to_csv("fin_df.csv")

# Parsing the clmmet dataset to add to corpus

In [83]:
fl = r'C:\Users\rhira\Documents\TextAge\plain\CLMET3_1_3_317.txt'

In [105]:
import os
import pandas as pd
from bs4 import BeautifulSoup

# Function to extract information from a file
def extract_info(file_path):
    with open(file_path, 'r',encoding="utf-8") as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')

    id_tag = soup.find('id')
    genre_tag = soup.find('genre')
    year_tag = soup.find('year')
    title_tag = soup.find('title')
    author_tag = soup.find('author')

    id_value = id_tag.text if id_tag else None
    genre_value = genre_tag.text if genre_tag else None
    year_value = year_tag.text if year_tag else None
    title_value = title_tag.text if title_tag else None
    author_value = author_tag.text if author_tag else None

    paragraphs = soup.find_all('p')
    p_content = '\n'.join(paragraph.text.strip() for paragraph in paragraphs)

    output_dict = {
        'text': p_content,
        'category': genre_value,
        'date': year_value,
        'title': title_value,
        'author': author_value
    }

    return output_dict

# Folder containing the HTML files
folder_path = r'C:\Users\rhira\Documents\TextAge\plain'

# List to store dictionaries for each file
data_list = []

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Assuming the files have an HTML extension
        file_path = os.path.join(folder_path, filename)
        #print(file_path)
        try:
            file_data = extract_info(file_path)
            data_list.append(file_data)
        except:
            print(file_path)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data_list)

# Print or use the DataFrame as needed
print(df.head())


                                                text           category  date  \
0  A TREATISE Concerning the PRINCIPLES OF Human ...           Treatise  1710   
1  SERMON I. UPON HUMAN NATURE .\nROMANS xii .\n4...           Treatise  1726   
2  THE London Merchant : OR , THE HISTORY OF GEOR...              Drama  1731   
3  MARINA : A PLAY OF THREE ACTS .\nAs it is Acte...              Drama  1738   
4  THE FORTUNATE FOUNDLINGS .\nCHAP .\nI .\nConta...  Narrative fiction  1744   

                                               title                 author  
0  A treatise concerning the principles of human ...       Berkeley, George  
1                     Human nature and other sermons         Butler, Joseph  
2                                The London merchant          Lillo, George  
3                                             Marina          Lillo, George  
4                           The fortunate foundlings  Haywood, Eliza Fowler  


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      333 non-null    object
 1   category  333 non-null    object
 2   date      333 non-null    object
 3   title     333 non-null    object
 4   author    333 non-null    object
dtypes: object(5)
memory usage: 13.1+ KB


In [110]:
df.head()

,text,category,date,title,author
0,A TREATISE Concerning the PRINCIPLES OF Human ...,Treatise,1710,A treatise concerning the principles of human ...,"Berkeley, George"
1,SERMON I. UPON HUMAN NATURE .\nROMANS xii .\n4...,Treatise,1726,Human nature and other sermons,"Butler, Joseph"
2,"THE London Merchant : OR , THE HISTORY OF GEOR...",Drama,1731,The London merchant,"Lillo, George"
3,MARINA : A PLAY OF THREE ACTS .\nAs it is Acte...,Drama,1738,Marina,"Lillo, George"
4,THE FORTUNATE FOUNDLINGS .\nCHAP .\nI .\nConta...,Narrative fiction,1744,The fortunate foundlings,"Haywood, Eliza Fowler"


In [111]:
df_old.head()

,Unnamed: 0,text,category,date,title,author
0,0,[The remedy against the troubles of temptation...,book.xml,1508.0,The-remedy-against-the-troubles-of-temptations,Richard-rolle
1,1,The ballade of ane right noble victorius & myg...,book.xml,1508.0,The-ballade-of-ane-right-noble-victorius,William-Dunbar
2,2,[The noble history of King Ponthus.]Ponthus et...,book.xml,1511.0,The-noble-history-of-King-Ponthus,Henry-watson
3,3,Here begynneth a dialogue betwene the comen se...,letter.xml,1530.0,dialogue-betwene-the-comen-secretary-and-Ielow...,Edward-Gosynhyll
4,4,"A compendious olde treatyse, shewynge howe tha...",book.xml,1530.0,A-compendious-olde-treatyse,Richard-ullerston


In [107]:
df_old = pd.read_csv("fin_df.csv")

df_fin2  = pd.concat([df_old, df], ignore_index=True)

In [108]:
df_fin2.to_csv("fin2.csv")

In [109]:
df_fin2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9633 entries, 0 to 9632
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9300 non-null   float64
 1   text        8199 non-null   object 
 2   category    8199 non-null   object 
 3   date        8199 non-null   object 
 4   title       8199 non-null   object 
 5   author      8185 non-null   object 
dtypes: float64(1), object(5)
memory usage: 451.7+ KB


# Court Case dataset - Using this was cancled as there were too many spelling errors in the short hand typing


In [115]:
import os
lst = os.listdir(r"C:\Users\rhira\Documents\TextAge\courtcase")
for fl in lst:
    splt = fl.split("_")
    yr = splt[0]
    aut = splt[1]
    title = splt[2]
    file_path = r"C:\Users\rhira\Documents\TextAge\courtcase" + "\\" + fl
    with open(file_path, 'r',encoding="utf-8") as file:
        content = file.read()
        print(content)
        break

4 Julii 1569 super libello ex parte Jane Clerck con Richard Churcher dat

Johannes Wilkins de Sora Winton glasier ubi moram fecit a nativitate &c. libere &c. testis &c. etatis xxv annos dicit quod dictam Clerck per viii annos et dict Churcher et uxor sua tam diu bene novit

Ad 1 articlum libelli refert se ad iura

Ad 2 deponit et dicit that abowt Ester last upon a working day (ut recolit) this deponent dwelling with Gillam glasier was requested by the saide Jane clerck to help her Mayde to the water with a buck. And when they came right over the saide Churchers dore with ther buck of clothes the saide churchers wiff had left wasshing. And was in her howse where upon James Godslens wiff his contest was standing upon a borde by the water redy to wash the clothes which this deponent & she brought forth. And then Churchers wiff came owte agayne from her howse with ii kettles And sayde to Mres Clerck then being ther / dost thow thrust my mayde from the wasshing place & imediatly went to the